In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
# Import necessary libraries
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.callbacks import EarlyStopping
import joblib  # For saving the scaler
import warnings
warnings.filterwarnings('ignore')

In [14]:
# Load your dataset
df = pd.read_csv('Data/stock_data.csv')
df = df.drop(columns=['Unnamed: 0'])
# Remove 'archive (1)\', or any other similar prefix, from the 'Stock' column
df['Stock'] = df['Stock'].str.replace(r'archive \(\d+\)\\', '', regex=True)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df.head()

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble,Stock
Date,,,,,,,,,,,,,,,
2007-11-27,MUNDRAPORT,EQ,440.00,770.00,1050.00,770.0,959.0,962.90,984.72,27294366,2.687719e+15,NaN,9859619.0,0.3612,ADANIPORTS
2007-11-28,MUNDRAPORT,EQ,962.90,984.00,990.00,874.0,885.0,893.90,941.38,4581338,4.312765e+14,NaN,1453278.0,0.3172,ADANIPORTS
2007-11-29,MUNDRAPORT,EQ,893.90,909.00,914.75,841.0,887.0,884.20,888.09,5124121,4.550658e+14,NaN,1069678.0,0.2088,ADANIPORTS
2007-11-30,MUNDRAPORT,EQ,884.20,890.00,958.00,890.0,929.0,921.55,929.17,4609762,4.283257e+14,NaN,1260913.0,0.2735,ADANIPORTS
2007-12-03,MUNDRAPORT,EQ,921.55,939.75,995.00,922.0,980.0,969.30,965.65,2977470,2.875200e+14,NaN,816123.0,0.2741,ADANIPORTS


In [15]:
df.keys()

Index(['Symbol', 'Series', 'Prev Close', 'Open', 'High', 'Low', 'Last',
       'Close', 'VWAP', 'Volume', 'Turnover', 'Trades', 'Deliverable Volume',
       '%Deliverble', 'Stock'],
      dtype='object')

In [16]:
# Preview the data
df.describe()


,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
count,235192.000000,235192.000000,235192.000000,235192.000000,235192.000000,235192.000000,235192.00000,2.351920e+05,2.351920e+05,1.203440e+05,2.191150e+05,219115.000000
mean,1266.196349,1267.759708,1286.581440,1247.488465,1266.388302,1266.554351,1267.13230,3.045903e+06,1.610138e+14,6.196427e+04,1.315098e+06,0.502997
std,2581.370320,2585.259609,2619.649216,2546.621396,2581.392543,2582.140942,2582.69998,7.333981e+06,3.298085e+14,6.866457e+04,2.831670e+06,0.190019
min,0.000000,8.500000,9.750000,8.500000,9.100000,9.150000,9.21000,3.000000e+00,1.047000e+07,1.100000e+01,5.000000e+00,0.023600
25%,274.300000,275.000000,279.500000,269.600000,274.400000,274.350000,274.69750,2.190095e+05,1.612816e+13,2.183400e+04,1.253830e+05,0.364700
50%,566.500000,567.025000,576.900000,556.500000,567.000000,566.700000,566.94000,1.010938e+06,6.832603e+13,4.406800e+04,5.017560e+05,0.511000
75%,1242.200000,1243.312500,1263.000000,1221.650000,1242.900000,1242.400000,1242.66250,3.019851e+06,1.863835e+14,7.893550e+04,1.452233e+06,0.638400
max,32861.950000,33399.950000,33480.000000,32468.100000,32849.000000,32861.950000,32975.24000,4.810589e+08,3.564334e+16,1.643015e+06,2.325307e+08,1.000000


In [17]:
data=df
# Define sequence length for LSTM
sequence_length = 60  # Use the past 60 days' data to predict the next day

# List of unique stock symbols
stock_symbols = data['Symbol'].unique()
stock_symbols

array(['MUNDRAPORT', 'ADANIPORTS', 'ASIANPAINT', 'UTIBANK', 'AXISBANK',
       'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJAUTOFIN', 'BAJFINANCE', 'BHARTI',
       'BHARTIARTL', 'BPCL', 'BRITANNIA', 'CIPLA', 'COALINDIA', 'DRREDDY',
       'EICHERMOT', 'GAIL', 'GRASIM', 'HCLTECH', 'HDFC', 'HDFCBANK',
       'HEROHONDA', 'HEROMOTOCO', 'HINDALC0', 'HINDALCO', 'HINDLEVER',
       'HINDUNILVR', 'ICICIBANK', 'INDUSINDBK', 'INFOSYSTCH', 'INFY',
       'IOC', 'ITC', 'JSWSTL', 'JSWSTEEL', 'KOTAKMAH', 'KOTAKBANK', 'LT',
       'MARUTI', 'M&M', 'NESTLEIND', 'NTPC', 'ONGC', 'POWERGRID',
       'RELIANCE', 'SBIN', 'SHREECEM', 'SUNPHARMA', 'TELCO', 'TATAMOTORS',
       'TISCO', 'TATASTEEL', 'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO',
       'UNIPHOS', 'UPL', 'SESAGOA', 'SSLT', 'VEDL', 'WIPRO', 'ZEETELE',
       'ZEEL'], dtype=object)

In [18]:
# Create a directory to save models and predictions
import os
if not os.path.exists('saved_models'):
    os.makedirs('saved_models')

if not os.path.exists('predictions'):
    os.makedirs('predictions')

# Function to prepare data for LSTM
def prepare_data(stock_data):
    # Select columns related to stock price and volume (features for prediction)
    data = stock_data[['Prev Close', 'Open', 'High', 'Low', 'Last', 'Close', 'Volume']]
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    return scaled_data, scaler

# Function to create sequences for LSTM
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length, 5])  # Predicting 'Close' price (index 5 corresponds to 'Close')
    return np.array(X), np.array(y)

# Function to train the model and predict for a given stock symbol
def train_and_predict(stock_symbol, df, seq_length=60):
    # Filter data for the selected stock
    stock_data = df[df['Symbol'] == stock_symbol]

    # Prepare data for LSTM
    scaled_data, scaler = prepare_data(stock_data)
    X, y = create_sequences(scaled_data, seq_length)

    # Split the data into training and test sets (80% training, 20% testing)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Build LSTM Model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))  # Output layer for Close price

    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32)

    # Save the trained model
    model.save(f'saved_models/{stock_symbol}_model.h5')
    print(f"Model for {stock_symbol} saved.")

    # Make predictions on the test set
    predictions = model.predict(X_test)

    # Inverse transform predictions and actual values to get the original scale
    predictions = scaler.inverse_transform(np.concatenate((np.zeros((predictions.shape[0], scaled_data.shape[1]-1)), predictions), axis=1))[:, -1]
    y_test_actual = scaler.inverse_transform(np.concatenate((np.zeros((y_test.shape[0], scaled_data.shape[1]-1)), y_test.reshape(-1, 1)), axis=1))[:, -1]

    # Save predictions to a CSV file
    prediction_df = pd.DataFrame({
        'Actual Close': y_test_actual,
        'Predicted Close': predictions
    })
    prediction_df.to_csv(f'predictions/{stock_symbol}_predictions.csv', index=False)
    print(f"Predictions for {stock_symbol} saved.")

    # Plot the actual vs predicted closing prices
    plt.figure(figsize=(14,7))
    plt.plot(y_test_actual, color='blue', label='Actual Closing Price')
    plt.plot(predictions, color='red', label='Predicted Closing Price')
    plt.title(f'{stock_symbol} Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()

    # Save the plot as an image file
    plt.savefig(f'predictions/{stock_symbol}_actual_vs_predicted.png')
    plt.close()  # Close the plot to avoid overlapping plots for next stock
    print(f"Plot for {stock_symbol} saved.")

# Iterate over all stock symbols and predict, saving models and predictions
for symbol in stock_symbols:
    print(f"Training and predicting for: {symbol}")
    train_and_predict(symbol, df)

Training and predicting for: MUNDRAPORT
Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 0.0347
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0061
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0045
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0043
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0047
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0029
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0035
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0033
Epoch 9/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0029
Epoch 10/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0038


Model for MUNDRAPORT saved.
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step
Predictions for MUNDRAPORT saved.
Plot for MUNDRAPORT saved.
Training and predicting for: ADANIPORTS
Epoch 1/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.0092
Epoch 2/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0010
Epoch 3/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0010
Epoch 4/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 8.8189e-04
Epoch 5/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 7.9095e-04
Epoch 6/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 7.9690e-04
Epoch 7/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 8.1929e-04
Epoch 8/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 7.3824e-04
Epoch 9/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 7.6065e-04
Epoch 10/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 7.6080e-04


Model for ADANIPORTS saved.
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Predictions for ADANIPORTS saved.
Plot for ADANIPORTS saved.
Training and predicting for: ASIANPAINT
Epoch 1/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - loss: 0.0116
Epoch 2/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.0022
Epoch 3/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.0016
Epoch 4/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 0.0018
Epoch 5/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.0014
Epoch 6/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 8.8563e-04
Epoch 7/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.0012
Epoch 8/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0011
Epoch 9/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.0011
Epoch 10/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 9.2710e-04


Model for ASIANPAINT saved.
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
Predictions for ASIANPAINT saved.
Plot for ASIANPAINT saved.
Training and predicting for: UTIBANK
Epoch 1/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 0.0093
Epoch 2/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 6.9035e-04
Epoch 3/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 6.1134e-04
Epoch 4/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 5.4654e-04
Epoch 5/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 4.1204e-04
Epoch 6/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.6187e-04
Epoch 7/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 4.0251e-04
Epoch 8/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 3.6029e-04
Epoch 9/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 4.8224e-04
Epoch 10/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 3.4440e-04


Model for UTIBANK saved.
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step
Predictions for UTIBANK saved.
Plot for UTIBANK saved.
Training and predicting for: AXISBANK
Epoch 1/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - loss: 0.0111
Epoch 2/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0029
Epoch 3/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0036
Epoch 4/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0030
Epoch 5/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0038
Epoch 6/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0022
Epoch 7/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0029
Epoch 8/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0027
Epoch 9/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0026
Epoch 10/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0017


...

Model for SSLT saved.
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step
Predictions for SSLT saved.
Plot for SSLT saved.
Training and predicting for: VEDL
Epoch 1/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 0.0674
Epoch 2/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0059
Epoch 3/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0048
Epoch 4/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0046
Epoch 5/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0045
Epoch 6/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0041
Epoch 7/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0037
Epoch 8/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0035
Epoch 9/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0040
Epoch 10/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0038


Model for VEDL saved.
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step
Predictions for VEDL saved.
Plot for VEDL saved.
Training and predicting for: WIPRO
Epoch 1/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 0.0013
Epoch 2/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 2.5730e-04
Epoch 3/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 1.9782e-04
Epoch 4/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 1.9800e-04
Epoch 5/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 1.7387e-04
Epoch 6/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 2.0005e-04
Epoch 7/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 1.5503e-04
Epoch 8/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 1.4094e-04
Epoch 9/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 2.2987e-04
Epoch 10/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 1.3018e-04


Model for WIPRO saved.
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
Predictions for WIPRO saved.
Plot for WIPRO saved.
Training and predicting for: ZEETELE
Epoch 1/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - loss: 0.0066
Epoch 2/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 8.2820e-04
Epoch 3/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 6.1949e-04
Epoch 4/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 4.0500e-04
Epoch 5/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 4.4921e-04
Epoch 6/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 4.8833e-04
Epoch 7/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 4.6734e-04
Epoch 8/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 4.3631e-04
Epoch 9/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 4.7039e-04
Epoch 10/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 4.5375e-04


Model for ZEETELE saved.
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step
Predictions for ZEETELE saved.
Plot for ZEETELE saved.
Training and predicting for: ZEEL
Epoch 1/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - loss: 0.0282
Epoch 2/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0031
Epoch 3/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0027
Epoch 4/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0026
Epoch 5/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0025
Epoch 6/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0022
Epoch 7/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0023
Epoch 8/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0021
Epoch 9/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0021
Epoch 10/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0020


Model for ZEEL saved.
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
Predictions for ZEEL saved.
Plot for ZEEL saved.
